Следећа свеска је аутоматски генерисана од стране GitHub Copilot Chat-а и намењена је само за почетно подешавање


# Увод у инжењеринг упита

Инжењеринг упита је процес осмишљавања и оптимизације упита за задатке обраде природног језика. Овај процес подразумева одабир одговарајућих упита, подешавање њихових параметара и процену њихових перформанси. Инжењеринг упита је кључан за постизање високе тачности и ефикасности код NLP модела. У овом одељку ћемо истражити основе инжењеринга упита користећи OpenAI моделе за експериментисање.


### Вежба 1: Токенизација
Истражите токенизацију користећи tiktoken, брзи отворени токенизатор из OpenAI
Погледајте [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) за још примера.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Вежба 2: Потврдите подешавање OpenAI API кључа

Покрените доњи код да бисте проверили да ли је ваш OpenAI крајњи сервис исправно подешен. Код само покушава један једноставан основни упит и проверава одговор. Унос `oh say can you see` треба да се настави отприлике са `by the dawn's early light..`


In [ ]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

## Updated
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

### Вежба 3: Измишљотине
Истражите шта се дешава када замолите LLM да врати одговоре на упит о теми која можда не постоји, или о темама о којима можда нема сазнања јер су ван његовог унапред обученог скупа података (нпр. новије теме). Погледајте како се одговор мења ако пробате другачији упит или други модел.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

### Вежба 4: На основу инструкција  
Користи променљиву "text" да поставиш главни садржај  
и променљиву "prompt" да даш инструкцију која се односи на тај главни садржај.

Овде тражимо од модела да сажме текст тако да га разуме ученик другог разреда.


In [ ]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

### Вежба 5: Комплексни захтеви
Пробајте захтев који садржи поруке система, корисника и асистента
Систем поставља контекст асистента
Поруке корисника и асистента обезбеђују контекст за разговор у више корака

Обратите пажњу како је личност асистента постављена на „саркастичну“ у системском контексту.
Пробајте да користите другачији контекст личности. Или пробајте другачији низ улазних/излазних порука


In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

### Вежба: Истражите своју интуицију
Горњи примери вам дају шаблоне које можете користити за креирање нових упита (једноставних, сложених, инструкција итд.) – пробајте да осмислите друге вежбе како бисте истражили неке од осталих идеја о којима смо причали, као што су примери, наговештаји и још много тога.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални људски превод. Не сносимо одговорност за било каква неспоразума или погрешна тумачења настала коришћењем овог превода.
